In [ ]:
# Extracción de la inforamción de los ligandos cocristalizados con la proteína CDK2

In [2]:
import pandas as pd
import numpy as np
import glob

In [3]:
prot_name = 'cdk2'

In [6]:
# Cargamos el dataframe principal de la proteína
# Este es el DF que no tiene aun afgregados los valores AUC de las conformaciones
df_prot_labels = pd.read_json('./B_DATOS/TABLA_MTDATA_CDK2_402_crys_LIGS_INFO_LABELS.json')

In [15]:
# Identificación de los inhibidores
# Seleccionamos las conformaciones que posean algun inhibidor
filtro_inh = df_prot_labels.Inhib != '' 
df_ligs_crys = df_prot_labels[filtro_inh]

In [16]:
# Número total de conformaciones con algún ligando en el sitio activo
df_ligs_crys.shape

(367, 15)

In [18]:
# Identificamos el número de ligandos únicos
ligs_crys_prot = set(df_ligs_cdk2.Inhib)
len(ligs_crys_prot )

317

In [125]:
# Web scrapping de los ligandos
import urllib.request
from bs4 import BeautifulSoup
from chembl_webresource_client.new_client import new_client
import pubchempy

In [39]:
URL_PDB = 'http://www.rcsb.org/ligand'

In [132]:
# Generamos un diccionario con 
dic_ligs_crys_prot = {}

for ligando in ligs_crys_prot:
    url_lig = F'{URL_PDB}/{ligando}'
    print(F'Procesando al ligando: {ligando}\n')
    # Extraemos los datos del html del ligando
    lig_data = urllib.request.urlopen( url_lig ).read().decode()
    # Minamos el html con BSoup para obtener el InChIKey
    # InChIKey
    soup = BeautifulSoup(lig_data)
    inchi_key = soup.find("tr", {"id": "chemicalInChIKey"})
    inchi_key_str = inchi_key.find('td').text
    # InChI
    inchi = soup.find("tr", {"id": "chemicalInChI"})
    inchi_str = inchi.find('td').text
    # isomeric SMILE (por si no se encuentra en la API de PbCh)
    smiles = soup.find("tr", {"id": "chemicalIsomeric"})
    smiles_str = smiles.find('td').text
    # Peso molecular
    mw = soup.find("tr", {"id": "chemicalMolecularWeight"})
    mw_str = mw.find('td').text
    # **** CHEMBL ***
    # Procedemos a extraer los datos del Chembl
    molecule = new_client.molecule
    try:
        chblMol_info = molecule.get(inchi_key_str)
        chembl_id = chblMol_info['molecule_chembl_id']
    except:
        print(F'No se pudo obtener infor de CEHMBL: {ligando}')
        print(inchi_str + '\n')
        chblMol_info = None
        chembl_id = None
    # *** PUCHEM ***
    try:
        pbch_mol = pubchempy.get_compounds(inchi_str, 'inchi')[0]
        pbch_cid = pbch_mol.cid
        pbch_data = pbch_mol.to_dict()
        canonical_smiles = pbch_mol.canonical_smiles
    except:
        print(F'Fallo con datos PUBCHEM: {ligando}')
        pbch_mol = None
        pbch_cid = None
        pbch_data = None
        canonical_smiles = None
    # Creamos la entrada para el diccionario
    # Primero el diccionario del ligando
    dic_lig = {'inchi_key': inchi_key_str,
               'inchi': inchi_str,
               'molecular_weight': float(mw_str),
               'canonical_smiles': canonical_smiles,
               'isomeric_similes': smiles_str,
               'chembl_id': chembl_id,
               'chembl_data': chblMol_info,
               'pubchem_id': pbch_cid,
               'pubchem_data': pbch_data,}
    # Anidamos el dict  del ligando al dic principal
    dic_ligs_crys_prot[ligando] = dic_lig

Procesando al ligando: L0F

Procesando al ligando: T3C

Procesando al ligando: HGW

No se pudo obtener infor de CEHMBL: HGW
InChI=1S/C4H3IN2/c5-4-1-6-3-7-2-4/h1-3H

Procesando al ligando: X88

Procesando al ligando: UN4

Procesando al ligando: IM9

Procesando al ligando: HH5

No se pudo obtener infor de CEHMBL: HH5
InChI=1S/C5H4IN5/c6-3-2-4(9-1-8-2)11-5(7)10-3/h1H,(H3,7,8,9,10,11)

Procesando al ligando: T2A

Procesando al ligando: 404

Procesando al ligando: 0BY

Procesando al ligando: 2KD

Procesando al ligando: 48K

Procesando al ligando: EZV

Procesando al ligando: DT1

Procesando al ligando: QQ2

No se pudo obtener infor de CEHMBL: QQ2
InChI=1S/C24H27ClN6O2/c1-17-4-9-21(25)22(12-17)31(11-10-26)24-13-23(27-16-28-24)29-18-5-7-20(8-6-18)33-15-19(32)14-30(2)3/h4-9,12-13,16,19,32H,11,14-15H2,1-3H3,(H,27,28,29)/t19-/m1/s1

Procesando al ligando: 0BX

Procesando al ligando: Z04

No se pudo obtener infor de CEHMBL: Z04
InChI=1S/C16H19N7O4/c17-15(24)11-5-14(23(25)26)13(22-1-3-27-4-2-22)6-1

Procesando al ligando: 09K

Procesando al ligando: Z30

No se pudo obtener infor de CEHMBL: Z30
InChI=1S/C12H20N4O/c1-12(2,3)7-4-5-9-8(6-7)10(16-15-9)11(17)14-13/h7H,4-6,13H2,1-3H3,(H,14,17)(H,15,16)/t7-/m1/s1

Procesando al ligando: RJI

No se pudo obtener infor de CEHMBL: RJI
InChI=1S/C11H9N3O4/c12-8(15)7-9(16)13-11(18)14(10(7)17)6-4-2-1-3-5-6/h1-5,7H,(H2,12,15)(H,13,16,18)/t7-/m0/s1

Procesando al ligando: 7YG

Procesando al ligando: D31

Procesando al ligando: X62

No se pudo obtener infor de CEHMBL: X62
InChI=1S/C12H9Cl2N5O3/c13-8-2-9(7(11(15)20)1-10(8)19(21)22)16-3-6-4-17-12(14)18-5-6/h1-2,4-5,16H,3H2,(H2,15,20)

Procesando al ligando: X6A

No se pudo obtener infor de CEHMBL: X6A
InChI=1S/C12H11ClN6O3/c13-8-2-9(7(11(14)20)1-10(8)19(21)22)16-3-6-4-17-12(15)18-5-6/h1-2,4-5,16H,3H2,(H2,14,20)(H2,15,17,18)

Procesando al ligando: 3TI

Procesando al ligando: PO5

No se pudo obtener infor de CEHMBL: PO5
InChI=1S/C25H30N8O3/c1-31(2)16-19(34)17-36-20-5-3-18(4-6-20)28-25-26-10-9-22(29-25)

Procesando al ligando: LS1

Procesando al ligando: 18Z

No se pudo obtener infor de CEHMBL: 18Z
InChI=1S/C18H25N9O3/c19-17(28)13-7-16(27(29)30)15(22-3-6-26-4-1-21-2-5-26)8-14(13)23-9-12-10-24-18(20)25-11-12/h7-8,10-11,21-23H,1-6,9H2,(H2,19,28)(H2,20,24,25)

Procesando al ligando: C94

Procesando al ligando: X0A

No se pudo obtener infor de CEHMBL: X0A
InChI=1S/C15H12ClN5O/c16-9-6-7-12(22)11(8-9)13-19-14(17)21-15(20-13)18-10-4-2-1-3-5-10/h1-8,22H,(H3,17,18,19,20,21)

Procesando al ligando: TJF

Procesando al ligando: FAP

No se pudo obtener infor de CEHMBL: FAP
InChI=1S/C21H23F2N5O2/c1-28(2)11-15(29)12-30-16-8-6-14(7-9-16)26-19-10-20(25-13-24-19)27-21-17(22)4-3-5-18(21)23/h3-10,13,15,29H,11-12H2,1-2H3,(H2,24,25,26,27)/t15-/m0/s1

Procesando al ligando: A28

Procesando al ligando: X42

Procesando al ligando: N41

Procesando al ligando: 56H

Procesando al ligando: 04Z

Procesando al ligando: 6SC

Procesando al ligando: U55

No se pudo obtener infor de CEHMBL: U55
InChI=1S/C10H11N5O2S/c11-

Procesando al ligando: F18

No se pudo obtener infor de CEHMBL: F18
InChI=1S/C9H10N6O/c10-8-7(9(11)15-14-8)13-12-5-1-3-6(16)4-2-5/h1-4,16H,(H5,10,11,14,15)/b13-12-

Procesando al ligando: Z71

No se pudo obtener infor de CEHMBL: Z71
InChI=1S/C16H13F2N5O3S2/c17-8-5-9(18)7-11(6-8)21-15(24)13-14(19)23-16(27-13)22-10-1-3-12(4-2-10)28(20,25)26/h1-7H,19H2,(H,21,24)(H,22,23)(H2,20,25,26)

Procesando al ligando: 628

Procesando al ligando: LZ9

Procesando al ligando: 12Z

Procesando al ligando: Z31

Procesando al ligando: 6AF

No se pudo obtener infor de CEHMBL: 6AF
InChI=1S/C17H21BrN4O3S/c1-10(23)11(2)25-16-15(18)9-20-17(22-16)21-12-3-5-13(6-4-12)26(19,24)14-7-8-14/h3-6,9-11,14,19,23H,7-8H2,1-2H3,(H,20,21,22)/t10-,11-,26+/m1/s1

Procesando al ligando: T9N

Procesando al ligando: CT7

No se pudo obtener infor de CEHMBL: CT7
InChI=1S/C13H11ClN4O2S/c1-21(19,20)10-4-2-9(3-5-10)16-13-8-11(14)17-12-6-7-15-18(12)13/h2-8,16H,1H3

Procesando al ligando: 292

Procesando al ligando: X35

Procesando al l

In [ ]:
# Ejemplo de visualización
